# Online Shoppers Purchasing Intention Prediction
Authors: Julian Daduica, Stephanie Ta, and Wai Ming Wong

In [1]:
from ucimlrepo import fetch_ucirepo # raw data is from this package
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

## Summary

## Introduction

The growth of online shopping or e-commerce has completely changed how people shop. Online shopping provides the convenience of exploring many different online stores effortlessly from their homes. This gives people more freedom over their time and choices. With this, retail e-commerce sales are estimated to exceed 4.1 trillion U.S. dollars worldwide in 2024 from roughly 2.7 billion online shoppers (Taheer, 2024; Commerce, 2024). In an evergrowing consumerism society, it is important to understand consumers’ behaviours in addition to their intentions. This can allow businesses to optimize the online shopping experience and maximize revenue in such a massive industry. When shopping in person, a store employee may find it easy to determine a person’s purchasing intention through various social cues. However, while shopping online, companies and businesses find it much more difficult to decide on the intentions of their customers. Businesses need to find solutions from data on user interactions such as page clicks, time spent on pages, time of day or year, and much more. With the evergrowing increase in website traffic, businesses must distinguish between visitors with strong purchasing intentions and those who are simply browsing. 

Machine learning is a powerful tool we can utilize to analyze and predict online shoppers purchasing intentions based on behavioural and interaction data. Using machine learning techniques, we can use algorithms and computation to analyze various features such as bounce rates, visitor type, time of year, and many others to identify patterns which can help predict purchasing intention. In this study, we aim to use a machine learning algorithm to predict online shoppers purchasing intentions. This will allow us to extract meaningful insights from user data. In such a lucrative field, determining purchasing intentions is vital to these companies and businesses for increasing revenue. This can help companies and businesses find optimal sales and marketing techniques, or personalize each customer experience on their website. 

## Methods

### Data

### Analysis

## Results and Discussion

In [2]:
#Dataset importing script from UCI ML Repository
# fetch dataset 
online_shoppers_purchasing_intention_dataset = fetch_ucirepo(id=468) 

# data (as pandas dataframes) and save it as csv
X = online_shoppers_purchasing_intention_dataset.data.features 
y = online_shoppers_purchasing_intention_dataset.data.targets
df = pd.concat([X, y], axis=1)
df.to_csv("../data/raw/raw_df.csv")

# variable information 
print(online_shoppers_purchasing_intention_dataset.variables) 

# split the training set and testing set and save them as csv files
train_df, test_df = train_test_split(df, test_size=0.3, random_state=123)
train_df.to_csv("../data/processed/train_df.csv")
test_df.to_csv("../data/processed/test_df.csv")

# split X, y in the training set and testing set
X_train = train_df.drop(columns=["Revenue"])
X_test = test_df.drop(columns=["Revenue"])
y_train = train_df["Revenue"]
y_test = test_df["Revenue"]


                       name     role         type demographic description  \
0            Administrative  Feature      Integer        None        None   
1   Administrative_Duration  Feature      Integer        None        None   
2             Informational  Feature      Integer        None        None   
3    Informational_Duration  Feature      Integer        None        None   
4            ProductRelated  Feature      Integer        None        None   
5   ProductRelated_Duration  Feature   Continuous        None        None   
6               BounceRates  Feature   Continuous        None        None   
7                 ExitRates  Feature   Continuous        None        None   
8                PageValues  Feature      Integer        None        None   
9                SpecialDay  Feature      Integer        None        None   
10                    Month  Feature  Categorical        None        None   
11         OperatingSystems  Feature      Integer        None        None   

In [3]:
# begin exploratory data analysis
train_df.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000
mean,2.318851,80.035963,0.496582,33.735985,31.506546,1179.548652,0.022252,0.043180,5.765987,0.063330,2.129765,2.353261,3.150852,4.071371
std,3.326228,173.132521,1.244019,138.995400,44.119701,1895.590842,0.048634,0.048648,18.215382,0.202414,0.925164,1.727358,2.408261,4.011918
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,182.208333,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.000000,0.000000,0.000000,18.000000,593.701980,0.003077,0.025466,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.115833,0.000000,0.000000,37.000000,1439.177083,0.017124,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,16.000000,2549.375000,584.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8631 entries, 2476 to 3582
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           8631 non-null   int64  
 1   Administrative_Duration  8631 non-null   float64
 2   Informational            8631 non-null   int64  
 3   Informational_Duration   8631 non-null   float64
 4   ProductRelated           8631 non-null   int64  
 5   ProductRelated_Duration  8631 non-null   float64
 6   BounceRates              8631 non-null   float64
 7   ExitRates                8631 non-null   float64
 8   PageValues               8631 non-null   float64
 9   SpecialDay               8631 non-null   float64
 10  Month                    8631 non-null   object 
 11  OperatingSystems         8631 non-null   int64  
 12  Browser                  8631 non-null   int64  
 13  Region                   8631 non-null   int64  
 14  TrafficType              8

In [5]:
corr_df = (
    train_df
    .corr('spearman', numeric_only=True)
    .abs()                      
    .stack()                   
    .reset_index(name='corr')
    .query(('level_0 < level_1')))  
corr_df

plot1 = alt.Chart(corr_df).mark_rect().encode(
    x=alt.X('level_0:N', title='Feature 1'),
    y=alt.Y('level_1:N', title='Feature 2'),
    size=alt.Size('corr:Q', title='Correlation Strength'),
    color=alt.Color('corr:Q'),
    tooltip=['level_0', 'level_1', 'corr']
).properties(
    width=600,
    height=600,
    title="Correlation Heatmap Between all Features"
)

plot1

alt.Chart(...)

Figure 1. Correlation plot between all features in dataset

In [6]:
# create baseline model to compare final model to
dummy_classifier = DummyClassifier()
dummy_classifier.fit(X_train, y_train)
dummy_cv_scores = pd.DataFrame(
    cross_validate(dummy_classifier, X_train, y_train, cv = 5, return_train_score = True))
mean_dummy_validation_accuracy = dummy_cv_scores['test_score'].mean()
mean_dummy_validation_accuracy

np.float64(0.8494960081213042)

In [7]:
# lists of each type of feature
numeric_cols = ['Administrative', 'Administrative_Duration',
                'Informational', 'Informational_Duration',
                'ProductRelated', 'ProductRelated_Duration',
                'BounceRates', 'ExitRates',
                'PageValues', 'SpecialDay']
categorical_cols = ['Weekend', 'OperatingSystems',
                    'Browser', 'Region',
                    'TrafficType', 'VisitorType']
ordinal_cols = ['Month']

In [8]:
# make preproccessor, note imputation is not needed since there are no null values in the data set
month_levels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

preprocessor = make_column_transformer(
    (StandardScaler(), numeric_cols),
    (OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore'), categorical_cols),
    (OrdinalEncoder(categories=[month_levels]), ordinal_cols)
)

In [9]:
# make pipeline including preprocessor and logistic regression model
log_reg_pipe = make_pipeline(
    preprocessor, LogisticRegression(max_iter=2000, random_state=123)
)

In [10]:
# tune hyperparameter C of the logistic regression model
param_grid = {
    "logisticregression__C": loguniform(1e-3, 1e3),
}

random_search = RandomizedSearchCV(
    log_reg_pipe,
    param_grid,
    n_iter=100,
    verbose=1,
    n_jobs=-1,
    random_state=123,
    return_train_score=True,
)

random_search.fit(X_train, y_train)

print("Best hyperparameter value: ", random_search.best_params_)
print("Best score: %0.3f" % (random_search.best_score_))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/juliandaduica/miniforge3/envs/online-shopping-prediction/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/juliandaduica/miniforge3/envs/online-shopping-prediction/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/juliandaduica/miniforge3/envs/online-shopping-prediction/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/juliandaduica/miniforge3/envs/online-shopping-prediction/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in c

Best hyperparameter value:  {'logisticregression__C': np.float64(1.5463515822289584)}
Best score: 0.886


Cross Validation Results:
- the best C for logistic regression: 0.916
- validation accuracy score of best logistic regression model: 0.887
- validation accuracy score of dummy classifier: 0.849
- validation accuracy score of best logistic regression model is slightly (3.8%) better than the validation accuracy score of the dummy classifier


In [11]:
# score dummy and best logistic regression model on test set
dummy_test_score = dummy_classifier.score(X_test, y_test)
log_reg_test_score = random_search.score(X_test, y_test)


print("Dummy classifier test score: %0.3f" % dummy_test_score)
print("Best logistic regression model test score: %0.3f" % log_reg_test_score)

Dummy classifier test score: 0.835
Best logistic regression model test score: 0.875


/Users/juliandaduica/miniforge3/envs/online-shopping-prediction/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Testing results:
- test accuracy score of best logistic regression model: 0.875
- test accuracy score of dummy classifier: 0.835
- test accuracy score of best logisitic regression model is just a bit (4%) better than the test accuracy score of the dmmy classfier

In [12]:
# find weights of each feature
best_estimator = random_search.best_estimator_
feature_names = best_estimator['columntransformer'].get_feature_names_out() # get feature names
weights = best_estimator["logisticregression"].coef_ # get feature coefficients

feat_weights = pd.DataFrame(weights, columns = feature_names)
feat_weights = feat_weights.T.reset_index()
feat_weights = feat_weights.rename(columns={'index': 'feature', 0: "weight"})
feat_weights['feature'] = feat_weights['feature'].str.split('__', expand = True)[1]
feat_weights.sort_values(by='weight', ascending=False)


,feature,weight
8,PageValues,1.436186
30,Browser_12,0.725649
56,TrafficType_16,0.709339
17,OperatingSystems_7,0.553704
51,TrafficType_11,0.472214
...,...,...
53,TrafficType_13,-0.662553
16,OperatingSystems_6,-0.694014
55,TrafficType_15,-0.799662
7,ExitRates,-0.844008


In [13]:
# average the weights of each feature

feat_weights['overall_feature'] = feat_weights['feature'].str.split('_', expand = True)[0]

# absolute value the weights so that positive and negative ones don't cancel eachother out
feat_weights['absolute_value_weight'] = abs(feat_weights['weight'])

pd.DataFrame(feat_weights.groupby('overall_feature'
    ).mean(numeric_only=True
    ).sort_values('absolute_value_weight', ascending = False
    )['absolute_value_weight'])

,absolute_value_weight
overall_feature,
PageValues,1.436186
ExitRates,0.844008
TrafficType,0.316245
VisitorType,0.299497
OperatingSystems,0.278076
Browser,0.255822
Weekend,0.163676
ProductRelated,0.145906
Region,0.125874


For determining the target, the `ExitRates` and `PagesValues` features seem to be the most important.

## References

Commerce, S. (2024, May 14). 43 ECommerce statistics in 2024 (global and U.s. data). SellersCommerce. https://www.sellerscommerce.com/blog/ecommerce-statistics/

Taheer, F. (2024, September 8). Online shopping statistics: How many people shop online in 2024? OptinMonster. https://optinmonster.com/online-shopping-statistics/